# VacationPy
## Overview
This notebook analyzes filters through a list of more than 500+ randomly selected cities across the world, looking for those with ideal weather conditions. It uses the Google Places API to finds hotels close to those perfect locations so that I can plan my next vacation, and it uses Jupyter-gmaps to visualize these locations across a worldwide humidity map. 
#### Notes
* Had to enable some things (in Bash, after activating Python Data environment, before opening Jupyter):
    - jupyter nbextension enable --py gmaps
    - jupyter nbextension enable --py widgetsnbextension
* Keep an eye on API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [3]:
# Read Cities csv that was created in the WeatherPy notebook and store into a Pandas DataFrame
input_data_file = "output_data/cities.csv"
city_df = pd.read_csv(input_data_file)

# Drop the extra index column
city_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

### Humidity Heatmap

In [5]:
# Configure gmaps - access maps with unique API key
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = city_df[["Latitude","Longitude"]]
humidity = city_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add Heatmap layer to map
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [9]:
# Narrow down the cities to fit your ideal weather conditions.
# First turn these columns into floats so we can do numeric comparisons on them
city_df[["Max Temp", "Humidity", "Cloudiness","Wind Speed"]].astype(float)

# Narrow down the cities to fit weather conditions.
perfect_temp = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70)]
low_clouds = perfect_temp.loc[(perfect_temp["Cloudiness"] == 0)]
not_too_windy = low_clouds.loc[(low_clouds["Wind Speed"] < 10)]

# Drop any rows with null values ad Store into variable named hotel_df.
hotel_df = not_too_windy.dropna().set_index("City").reset_index()

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

### Add Hotels to Map


In [10]:
# Set parameters to do a nearby places search for hotels witin 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}
# Iterate through the df of cities with perfect weather and hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():

    # get latitude & longitude of each city
    lat = row["Latitude"]
    lng = row["Longitude"]
       
    # change the location parameter for each city
    params['location'] = f"{lat},{lng}"

    # assemble the url and call the nearby places search API
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"] = (response["results"][0]["name"])
    except: 
        pass
    
# Remove all of the rows from the hotel_df for cities that the API did not find hotels for
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]

In [12]:
hotel_df.head(12)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,moratuwa,6.7730,79.8816,77.00,78,0,9.22,LK,1612381205,Thotupola Lakeside
2,east london,-33.0153,27.9116,75.20,78,0,2.30,ZA,1612381180,Blue Lagoon Hotel
3,atar,20.5169,-13.0499,72.97,19,0,0.67,MR,1612381258,Odar kanawal
4,pisco,-13.7000,-76.2167,75.99,69,0,8.05,PE,1612381234,HOSTAL MIRAMAR
5,moa,20.6569,-74.9403,73.71,54,0,8.50,CU,1612381266,Islazul Miraflores
8,constitucion,-35.3333,-72.4167,71.71,50,0,9.48,CL,1612381264,Hotel Boutique Las Azucenas
10,ratnagiri,16.9833,73.3000,74.21,43,0,4.72,IN,1612381316,"Kohinoor Samudra Beach Resort, Ratnagiri"
11,ancud,-41.8697,-73.8203,71.60,56,0,6.91,CL,1612381346,Panamericana Hotel Ancud
12,kampot,10.6167,104.1833,71.60,88,0,2.30,KH,1612381360,Bamboo Bungalow


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, 
                            info_box_content = hotel_info)

# Add the layers to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))